In [ ]:
MAX_LEN = 256
bs = 32

In [ ]:
import pandas as pd
!pip install transformers
!pip install tokenizers

In [ ]:
train = '/content/NER_Irish_train.conll'
val = '/content/NER_Irish_validation.conll'
test = '/content/NER_Irish_test.conll'

In [ ]:
def read_conll_file(file_path):
    data = []
    current_sentence = []
    with open(file_path, "r", encoding="utf-8") as file:
        for line in file:
            line = line.strip()
            if line.startswith('-DOCSTART-'):
                continue
            if line:
                parts = line.split()
                word = parts[0]
                ner_label = parts[-1]
                current_sentence.append((word, ner_label))
            else:
                if current_sentence:
                    data.append(current_sentence)
                    current_sentence = []
    if current_sentence:
        data.append(current_sentence)
    return data

train_data = read_conll_file(train)
val_data = read_conll_file(val)
test_data = read_conll_file(test)

In [ ]:
print(train_data)

[[('\ufeffDáil', 'B-ORG'), ('Éireann', 'I-ORG'), ('06', 'O'), ('/', 'O'), ('07', 'O'), ('/', 'O'), ('2023', 'O')], [('Baineann', 'O'), ('mo', 'O'), ('cheist', 'O'), ('le', 'O'), ('cúrsaí', 'O'), ('tithíochta', 'O'), ('i', 'O'), ('nGaillimh', 'B-LOC'), ('-', 'O'), ('tá', 'O'), ('sé', 'O'), ('níos', 'O'), ('cirte', 'O'), ('easpa', 'O'), ('tithíochta', 'O'), ('i', 'O'), ('nGaillimh', 'B-LOC'), ('a', 'O'), ('rá', 'O'), ('-', 'O'), ('agus', 'O'), ('an', 'O'), ('tascfhórsa', 'O'), ('a', 'O'), ('bunaíodh', 'O'), ('breis', 'O'), ('agus', 'O'), ('ceithre', 'O'), ('bliana', 'O'), ('ó', 'O'), ('shin', 'O')], [('Ar', 'O'), ('dheis', 'O'), ('Dé', 'B-PER'), ('go', 'O'), ('raibh', 'O'), ('a', 'O'), ('anam', 'O')], [('Táim', 'O'), ('os', 'O'), ('comhair', 'O'), ('Dail', 'B-ORG'), ('Éireann', 'I-ORG')], [('Ba', 'O'), ('mhaith', 'O'), ('liom', 'O'), ('tréaslú', 'O'), ('leis', 'O'), ('an', 'O'), ('Teachta', 'O'), ('Andrews', 'B-PER'), ('as', 'O'), ('an', 'O'), ('obair', 'O'), ('atá', 'O'), ('déanta', 'O'

In [ ]:
print(val_data)

[[('Crothnófar', 'O'), ('Pól', 'B-PER'), ('nó', 'O'), ('ba', 'O'), ('úrlabhraí', 'O'), ('maith', 'O'), ('é', 'O'), ('ar', 'O'), ('Raidió', 'B-ORG'), ('na', 'I-ORG'), ('Gaeltachta', 'I-ORG'), ('agus', 'O'), ('na', 'O'), ('meáin', 'O'), ('eile', 'O'), ('ag', 'O'), ('cosaint', 'O'), ('na', 'O'), ('n-oifigí', 'O'), ('poist', 'O'), ('tuaithe', 'O'), (',', 'O'), ('gné', 'O'), ('am-tábhachtach', 'O'), ('de', 'O'), ('shaol', 'O'), ('sóisialta', 'O'), ('na', 'O'), ('ndaoine', 'O'), ('.', 'O')], [('Dá', 'O'), ('mba', 'O'), (',', 'O'), ('mar', 'O'), ('sin', 'O'), (',', 'O'), ('nach', 'O'), ('bhfaca', 'O'), ('an', 'O'), ('tSiúr', 'O'), ('Concepta', 'B-PER'), ('an', 'O'), ('tréadaí', 'O'), ('Eoin', 'B-PER'), ('Mac', 'I-PER'), ('Diarmada', 'I-PER'), ('ag', 'O'), ('bailiú', 'O'), ('leis', 'O'), ('ag', 'O'), ('an', 'O'), ('mbreacsholas', 'O'), (',', 'O'), ('níorbh', 'O'), ('aistí', 'O'), ('rud', 'O'), ('riamh', 'O'), ('ná', 'O'), ('an', 'O'), ('radharc', 'O'), ('a', 'O'), ('bhí', 'O'), ('le', 'O'), ('

In [ ]:
print(test_data)

[[('Mar', 'O'), ('a', 'O'), ('tchítear', 'O'), ('do', 'O'), ('Sheosamh', 'B-PER'), ('Mac', 'I-PER'), ('Grianna', 'I-PER'), ('é', 'O'), ('caithfidh', 'O'), ('an', 'O'), ('t-ealaíontóir', 'O'), ('an', 'O'), ('solas', 'O'), ('a', 'O'), ('thabhairt', 'O'), ('don', 'O'), ('saol', 'O'), ('agus', 'O'), ('diúltú', 'O'), ('do', 'O'), ('chathú', 'O'), ('sin', 'O'), ('na', 'O'), ('truaillíochta', 'O'), ('a', 'O'), ('chuireann', 'O'), ('an', 'O'), ('saol', 'O'), ('ina', 'O'), ('chosán', 'O'), ('.', 'O')], [('Dhiúltaigh', 'O'), ('Johnathan', 'B-PER'), ('Cape', 'I-PER'), ('agus', 'O'), ('cúpla', 'O'), ('foilsitheoir', 'O'), ('eile', 'O'), ('an', 'O'), ('leabhar', 'O'), ('a', 'O'), ('fhoilsiú', 'O'), ('agus', 'O'), ('bhí', 'O'), ('Naomi', 'B-PER'), ('le', 'O'), ('ceangal', 'O'), ('go', 'O'), ('raibh', 'O'), ('srian', 'O'), ('a', 'O'), ('chur', 'O'), ('ar', 'O'), ('a', 'O'), ('cuid', 'O'), ('tuairimíochta', 'O'), ('.', 'O')], [('Grianghraif', 'O'), ('le', 'O'), ('Maidhc', 'B-PER'), ('Ó', 'I-PER'), ('S

In [ ]:
import torch
from transformers import AutoTokenizer

# Enocde the labels
def encode_labels(sentence, tokenizer, label_map):
    tokens = []
    encoded_labels = []
    word_indices = []

    for word, label in sentence:
        word_tokens = tokenizer.tokenize(word)
        word_indices.append(len(tokens))  # Track the index of the original word
        tokens.extend(word_tokens)
        # Assign label to each sub-token
        for _ in word_tokens:
            encoded_labels.append(label_map[label])

    return tokens, encoded_labels, word_indices

def prepare_bert_inputs(data):
    tokenizer = AutoTokenizer.from_pretrained('bert-base-multilingual-cased')
    bert_inputs = []
    max_seq_length = 256  # Defining the maximum sequence length

    # Dictionary of labels mapped to integer representations
    label_map = {
        'O': 0,
        'B-PER': 1,
        'I-PER': 2,
        'B-LOC': 3,
        'I-LOC': 4,
        'B-ORG': 5,
        'I-ORG': 6
    }

    for sentence in data:
        tokens, encoded_labels, word_indices = encode_labels(sentence, tokenizer, label_map)

        # Padding the sequences
        tokens = ['[CLS]'] + tokens + ['[SEP]']
        padding_length = max_seq_length - len(tokens)
        tokens += [tokenizer.pad_token] * padding_length

        # 0 corresponds to 'O' tag
        encoded_labels = [0] + encoded_labels + [0]

        # Adjust encoded_labels based on original word indices
        adjusted_labels = [encoded_labels[idx] for idx in word_indices]
        adjusted_labels += [0] * (max_seq_length - len(adjusted_labels))

        # Converting the tokens to input IDs and creating attention mask
        input_ids = tokenizer.convert_tokens_to_ids(tokens)
        attention_mask = [1] * len(input_ids)

        # Padding the attention_mask
        padding_length = max_seq_length - len(attention_mask)
        attention_mask += [0] * padding_length

        # Converting adjusted_labels to tensor
        adjusted_labels = torch.tensor(adjusted_labels, dtype=torch.long)

        # Appending the data to bert_inputs in a dict format
        bert_inputs.append({
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': adjusted_labels
        })

    return bert_inputs

In [ ]:
train_inputs = prepare_bert_inputs(train_data)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

In [ ]:
print(train_data[12])
print(train_inputs[12])

[('Ní', 'O'), ('féidir', 'O'), ('linn', 'O'), ('muinín', 'O'), ('a', 'O'), ('bheith', 'O'), ('againn', 'O'), ('as', 'O'), ('an', 'O'), ('eolas', 'O'), ('a', 'O'), ('thagann', 'O'), ('ó', 'O'), ('mbord', 'O'), ('feidhmiúcháin', 'O'), ('Nike', 'B-ORG'), ('agus', 'O'), ('tá', 'O'), ('feidhmeannaigh', 'O'), ('an', 'O'), ('bhoird', 'O'), ('ag', 'O'), ('caitheamh', 'O'), ('go', 'O'), ('holc', 'O'), ('leis', 'O'), ('an', 'O'), ('bpobal', 'O'), ('.', 'O')]
{'input_ids': [101, 82020, 69494, 39758, 12361, 10245, 13188, 169, 170, 15543, 10237, 13123, 10115, 10146, 10151, 13934, 13983, 169, 108270, 88337, 273, 181, 56795, 34778, 51141, 10500, 42596, 29782, 10245, 68502, 12732, 30185, 34778, 51141, 41947, 16441, 26009, 10151, 170, 10758, 10835, 10162, 16942, 39270, 26900, 72611, 11783, 110516, 10350, 19790, 10151, 170, 13520, 17140, 119, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

# Testing Augmentation

In [ ]:
train_data[0:10]

[[('\ufeffDáil', 'B-ORG'),
  ('Éireann', 'I-ORG'),
  ('06', 'O'),
  ('/', 'O'),
  ('07', 'O'),
  ('/', 'O'),
  ('2023', 'O')],
 [('Baineann', 'O'),
  ('mo', 'O'),
  ('cheist', 'O'),
  ('le', 'O'),
  ('cúrsaí', 'O'),
  ('tithíochta', 'O'),
  ('i', 'O'),
  ('nGaillimh', 'B-LOC'),
  ('-', 'O'),
  ('tá', 'O'),
  ('sé', 'O'),
  ('níos', 'O'),
  ('cirte', 'O'),
  ('easpa', 'O'),
  ('tithíochta', 'O'),
  ('i', 'O'),
  ('nGaillimh', 'B-LOC'),
  ('a', 'O'),
  ('rá', 'O'),
  ('-', 'O'),
  ('agus', 'O'),
  ('an', 'O'),
  ('tascfhórsa', 'O'),
  ('a', 'O'),
  ('bunaíodh', 'O'),
  ('breis', 'O'),
  ('agus', 'O'),
  ('ceithre', 'O'),
  ('bliana', 'O'),
  ('ó', 'O'),
  ('shin', 'O')],
 [('Ar', 'O'),
  ('dheis', 'O'),
  ('Dé', 'B-PER'),
  ('go', 'O'),
  ('raibh', 'O'),
  ('a', 'O'),
  ('anam', 'O')],
 [('Táim', 'O'),
  ('os', 'O'),
  ('comhair', 'O'),
  ('Dail', 'B-ORG'),
  ('Éireann', 'I-ORG')],
 [('Ba', 'O'),
  ('mhaith', 'O'),
  ('liom', 'O'),
  ('tréaslú', 'O'),
  ('leis', 'O'),
  ('an', 'O'),
  ('

In [ ]:
import random

def augment_sentence_with_entities_v3(sentence, entity_pool, num_entities=1):
    augmented_sentence = sentence[:]  # Make a copy of the sentence to preserve the original data.

    # Determine insertion positions excluding the last position unless it's 'O'.
    insert_positions = [i for i, (_, label) in enumerate(sentence[:-1]) if label == 'O' and sentence[i+1][1] == 'O']
    if sentence[-1][1] == 'O':
        insert_positions.append(len(sentence)-1)  # Include the last position if it's 'O'.

    for _ in range(num_entities):
        if not insert_positions:
            break  # Exit if no suitable position is found.

        insert_position = random.choice(insert_positions)
        # Select a random 'B-' tag entity to insert.
        b_tag_entities = {k: v for k, v in entity_pool.items() if k.startswith('B-')}
        entity_label = random.choice(list(b_tag_entities.keys()))
        entity_to_insert = random.choice(b_tag_entities[entity_label])
        entity_tokens = entity_to_insert.split()

        # Insert the 'B-' entity at the chosen position.
        for token in entity_tokens:
            augmented_sentence.insert(insert_position + 1, (token, entity_label))
            insert_position += 1  # We uUpdate the position for the next token insertion.

        i_tag_tokens = []  # Initialise the list to avoid the NameError.
        # Optionally insert 'I-' tags following the 'B-' tag, ensuring logical consistency.
        i_tag_label = 'I' + entity_label[1:]  # Convert 'B-' tag to 'I-' tag.
        if i_tag_label in entity_pool and entity_pool[i_tag_label]:
            i_tag_entity = random.choice(entity_pool[i_tag_label])
            i_tag_tokens = i_tag_entity.split()
            for token in i_tag_tokens:
                # Ensure the insertion follows logical consistency rules.
                if insert_position < len(augmented_sentence) - 1 and (augmented_sentence[insert_position + 1][1] == 'O' or augmented_sentence[insert_position + 1][1].startswith('I-')):
                    augmented_sentence.insert(insert_position + 1, (token, i_tag_label))
                    insert_position += 1

        # Update insert positions to avoid clustering inserted entities.
        insert_positions = [pos for pos in insert_positions if pos < insert_position or pos > insert_position + len(entity_tokens) + len(i_tag_tokens)]

    return augmented_sentence



In [ ]:
entity_pool = {}
i = 0
while i < len(train_data):
  for word, label in train_data[i]:
    if label != 'O':
        if label not in entity_pool:
            entity_pool[label] = []
        if word not in entity_pool[label]:
            entity_pool[label].append(word)
  i += 1  # Increment i inside the while loop

print(entity_pool)
print(len(entity_pool['B-PER']))
print(len(entity_pool['I-PER']))
print(len(entity_pool['B-ORG']))
print(len(entity_pool['I-ORG']))
print(len(entity_pool['B-LOC']))
print(len(entity_pool['I-LOC']))



{'B-ORG': ['\ufeffDáil', 'Dail', 'Dáil', 'Fís', 'RTÉ', 'TG4', 'Údarás', 'Nike', 'an', 'Sky', 'Dála', 'Sinn', 'An', 'HSE', 'Raidió', 'Sláintecare', 'Chumann', 'gComhaontas', 'Fianna', 'Fine', 'd’Fhianna', 'Teach', 'Roinne', 'Cúla4', 'd’Údarás', 'DEIS', 'Chonradh', 'Massie', 'New', 'John', 'bhForas', '“Irish', 'VirginMedia', 'S4C', 'CLG', 'gCumann', 'Fórsa', 'SIPTU', 'INMO', 'Enable', 'Tusla', 'FSS', 'Thithe', 'Foras', 'gComhairle', 'CATT', 'Conradh', 'Glór', 'Gael', 'Údaras', 'gComhchoiste', 'Comhchoiste', 'Comhairle', 'Chomhairle', 'Rannóg', 'Cluichí', 'Cumann', 'Citylink', 'Coláiste', 'Seanad', 'Choláiste', 'Rialtas', 'Aontas', 'Gharda', 'Banc', 'Fhine', 'Fhianna', 'hArd-Chúirte', 'Oireachtas', 'Oireachtais', 'Grant', 'Choiste', 'Chomhchoiste', 'Roinn', "'Seirbhísí", 'Comhdháil', 'Cúldoras', 'Acadamh', 'FBI', 'Chlub', 'Tia', 'Oifig', 'INTRA', 'gColáiste', 'College', 'hÉireannaigh', 'hArm', 'BBC', 'Globalise', 'gCuntas', 'Thoghroinn', 'Náisiún', 'Faisean', 'Éire', 'Scoil', 'ECU', 'ANOI

In [ ]:
import random

def replace_and_adjust_entities(sentence, words_to_replace, entity_pool):
    updated_sentence = []
    for i, (word, label) in enumerate(sentence):
        if word in words_to_replace:
            # Choose a random 'B-PER' entity to replace
            b_per_entity = random.choice(entity_pool['B-PER']).split()
            for j, entity_word in enumerate(b_per_entity):
                # Use 'B-PER' for the first token, 'I-PER' for subsequent tokens
                new_label = 'I-PER' if j > 0 else 'B-PER'
                updated_sentence.append((entity_word, new_label))
        else:
            updated_sentence.append((word, label))

    # Adjust consecutive entities
    adjusted_sentence = []
    previous_label = None
    for word, label in updated_sentence:
        if label == 'B-PER' and previous_label == 'B-PER':
            adjusted_label = 'I-PER'
        else:
            adjusted_label = label
        adjusted_sentence.append((word, adjusted_label))
        previous_label = label

    return adjusted_sentence

# A simple example
sentence = [('Sí', 'O'), ('Sé', 'O'), ('to', 'O'), ('the', 'O'), ('store', 'O'), ('and', 'O'), ('Sé', 'O'), ('followed', 'O')]
words_to_replace = ['Sí', 'Sé']
updated_and_adjusted_sentence = replace_and_adjust_entities(sentence, words_to_replace, entity_pool)
print(updated_and_adjusted_sentence)


[('Padraic', 'B-PER'), ('Martin', 'I-PER'), ('to', 'O'), ('the', 'O'), ('store', 'O'), ('and', 'O'), ('Taoisigh', 'B-PER'), ('followed', 'O')]


In [ ]:
def adjust_consecutive_entities(sentence):
    adjusted_sentence = []
    previous_label = None  # Keep track of the previous label

    for word, label in sentence:
        if label == 'B-PER' and previous_label == 'B-PER':
            # Change to 'I-PER' if the previous label was also 'B-PER'
            adjusted_label = 'I-PER'
        else:
            adjusted_label = label

        adjusted_sentence.append((word, adjusted_label))
        previous_label = label  # Update the previous label for the next iteration

    return adjusted_sentence

# Example
sentence = [
    ('Micheal', 'B-PER'), ('Briain', 'B-PER'), ('to', 'O'), ('the', 'O'),
    ('store', 'O'), ('and', 'O'), ('Ard-Reachtaire', 'B-PER'), ('followed', 'O')
]
adjusted_sentence = adjust_consecutive_entities(sentence)
print(adjusted_sentence)


[('Micheal', 'B-PER'), ('Briain', 'I-PER'), ('to', 'O'), ('the', 'O'), ('store', 'O'), ('and', 'O'), ('Ard-Reachtaire', 'B-PER'), ('followed', 'O')]


In [ ]:
augmented_train_data_v3 = [augment_sentence_with_entities_v3(sentence, entity_pool, num_entities=1) for sentence in train_data]
#train_inputs_v2 = prepare_bert_inputs(augmented_train_data_v3, tokenizer, label_map)

print(train_data[35])
print(augmented_train_data_v3[35])

[('Tagann', 'O'), ('an', 'O'), ('chéad', 'O'), ('cheist', 'O'), ('ón', 'O'), ('Teachta', 'O'), ('Ó', 'B-PER'), ('Cuív', 'I-PER'), ('.', 'O')]
[('Tagann', 'O'), ('an', 'O'), ('chéad', 'O'), ('cheist', 'O'), ('ón', 'O'), ('Leabharlannaí', 'B-PER'), ('Behal', 'I-PER'), ('Teachta', 'O'), ('Ó', 'B-PER'), ('Cuív', 'I-PER'), ('.', 'O')]


# Testing on Data

In [ ]:
train_data[0]

[('\ufeffDáil', 'B-ORG'),
 ('Éireann', 'I-ORG'),
 ('06', 'O'),
 ('/', 'O'),
 ('07', 'O'),
 ('/', 'O'),
 ('2023', 'O')]

In [ ]:
augmented_train_data_v3 = [augment_sentence_with_entities_v3(sentence, entity_pool, num_entities=1) for sentence in train_data]
train_inputs_v3 = prepare_bert_inputs(augmented_train_data_v3)


print(train_data[0])
print(augmented_train_data_v3[0])
print(train_inputs_v3[2])

[('\ufeffDáil', 'B-ORG'), ('Éireann', 'I-ORG'), ('06', 'O'), ('/', 'O'), ('07', 'O'), ('/', 'O'), ('2023', 'O')]
[('\ufeffDáil', 'B-ORG'), ('Éireann', 'I-ORG'), ('06', 'O'), ('/', 'O'), ('07', 'O'), ('gCill', 'B-LOC'), ('Shádach', 'I-LOC'), ('/', 'O'), ('2023', 'O')]
{'input_ids': [101, 18484, 11034, 10291, 141, 10333, 11783, 40931, 169, 226, 84974, 11643, 22133, 156, 35899, 35636, 18812, 10147, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [ ]:
eval_inputs = prepare_bert_inputs(val_data)
test_inputs = prepare_bert_inputs(test_data)

In [ ]:
!pip install accelerate -U

In [ ]:
!pip install transformers[torch]

# Evaluation

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)

In [ ]:
tag_values = [
        'O',
        'B-PER',
        'I-PER',
        'B-LOC',
        'I-LOC',
        'B-ORG',
        'I-ORG'
]

tag_values.append("PAD")
tag2idx = {t: i for i, t in enumerate(tag_values)}

In [ ]:
tag2idx

{'O': 0,
 'B-PER': 1,
 'I-PER': 2,
 'B-LOC': 3,
 'I-LOC': 4,
 'B-ORG': 5,
 'I-ORG': 6,
 'PAD': 7}

In [ ]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from transformers import BertConfig

In [ ]:
MAX_LEN = 256
bs = 32

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()

In [ ]:
torch.cuda.get_device_name(0)

'NVIDIA L4'

In [ ]:
train_sent = []

for lst in augmented_train_data_v3:
  sent = []
  for words in lst:
    sent.append(words[0])
  train_sent.append(sent)

In [ ]:
val_sent = []

for lst in val_data:
  sent = []
  for words in lst:
    sent.append(words[0])
  val_sent.append(sent)

In [ ]:
test_sent = []

for lst in test_data:
  sent = []
  for words in lst:
    sent.append(words[0])
  test_sent.append(sent)

In [ ]:
train_labels = []

for lst in augmented_train_data_v3:
  lab = []
  for words in lst:
    lab.append(words[1])
  train_labels.append(lab)

In [ ]:
val_labels = []

for lst in val_data:
  lab = []
  for words in lst:
    lab.append(words[1])
  val_labels.append(lab)

In [ ]:
test_labels = []

for lst in test_data:
  lab = []
  for words in lst:
    lab.append(words[1])
  test_labels.append(lab)

In [ ]:
print(train_sent[1])

['Baineann', 'mo', 'cheist', 'le', 'gClár', 'Coire', 'cúrsaí', 'tithíochta', 'i', 'nGaillimh', '-', 'tá', 'sé', 'níos', 'cirte', 'easpa', 'tithíochta', 'i', 'nGaillimh', 'a', 'rá', '-', 'agus', 'an', 'tascfhórsa', 'a', 'bunaíodh', 'breis', 'agus', 'ceithre', 'bliana', 'ó', 'shin']


In [ ]:
print(train_labels[1])

['O', 'O', 'O', 'O', 'B-LOC', 'I-LOC', 'O', 'O', 'O', 'B-LOC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-LOC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


In [ ]:
def tokenize_and_preserve_labels(sentence, text_labels):
    tokenized_sentence = []
    labels = []

    for word, label in zip(sentence, text_labels):

        # Tokenise the word and count # of subwords the word is broken into
        tokenized_word = tokenizer.tokenize(word)
        n_subwords = len(tokenized_word)

        # Add the tokenised word to the final tokenised word list
        tokenized_sentence.extend(tokenized_word)

        # Add the same label to the new list of labels `n_subwords` times
        labels.extend([label] * n_subwords)

    return tokenized_sentence, labels

In [ ]:
tokenized_texts_and_labels = [
    tokenize_and_preserve_labels(sent, labs)
    for sent, labs in zip(train_sent, train_labels)
]

val_tokenized_texts_and_labels = [
    tokenize_and_preserve_labels(sent, labs)
    for sent, labs in zip(val_sent, val_labels)
]

test_tokenized_texts_and_labels = [
    tokenize_and_preserve_labels(sent, labs)
    for sent, labs in zip(test_sent, test_labels)
]

In [ ]:
tokenized_texts = [token_label_pair[0] for token_label_pair in tokenized_texts_and_labels]
labels = [token_label_pair[1] for token_label_pair in tokenized_texts_and_labels]

val_tokenized_texts = [token_label_pair[0] for token_label_pair in val_tokenized_texts_and_labels]
val_labels = [token_label_pair[1] for token_label_pair in val_tokenized_texts_and_labels]

test_tokenized_texts = [token_label_pair[0] for token_label_pair in test_tokenized_texts_and_labels]
test_labels = [token_label_pair[1] for token_label_pair in test_tokenized_texts_and_labels]

In [ ]:
print(tokenized_texts)
print(labels)

[['D', '##áil', 'Éireann', '06', '/', '07', 'g', '##C', '##ill', 'S', '##há', '##dach', '/', '202', '##3'], ['Bai', '##nea', '##nn', 'mo', 'che', '##ist', 'le', 'g', '##C', '##lá', '##r', 'Co', '##ire', 'c', '##úr', '##sa', '##í', 'ti', '##th', '##íocht', '##a', 'i', 'n', '##G', '##ail', '##lim', '##h', '-', 'tá', 'sé', 'níos', 'ci', '##rte', 'ea', '##sp', '##a', 'ti', '##th', '##íocht', '##a', 'i', 'n', '##G', '##ail', '##lim', '##h', 'a', 'rá', '-', 'agus', 'an', 'tas', '##c', '##f', '##h', '##ór', '##sa', 'a', 'buna', '##íodh', 'br', '##eis', 'agus', 'cei', '##th', '##re', 'bli', '##ana', 'ó', 'shi', '##n'], ['Ar', 'dhe', '##is', 'D', '##é', 'go', 'raibh', 'a', 'Á', '##isit', '##he', '##oir', 'S', '##lá', '##inte', 'ana', '##m'], ['Tá', '##im', 'os', 'California', 'h', '##E', '##orp', '##a', 'com', '##hair', 'Dai', '##l', 'Éireann'], ['Ba', 'm', '##hai', '##th', 'li', '##om', 'tr', '##éa', '##sl', '##ú', 'leis', 'an', 'Tea', '##chta', 'Andrews', 'as', 'an', 'oba', '##ir', 'M', '##há

In [ ]:
input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts],
                          maxlen=MAX_LEN, dtype="long", value=0.0,
                          truncating="post", padding="post")

val_input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in val_tokenized_texts],
                          maxlen=MAX_LEN, dtype="long", value=0.0,
                          truncating="post", padding="post")

test_input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in test_tokenized_texts],
                          maxlen=MAX_LEN, dtype="long", value=0.0,
                          truncating="post", padding="post")

In [ ]:
tags = pad_sequences([[tag2idx.get(l) for l in lab] for lab in labels],
                     maxlen=MAX_LEN, value=tag2idx["PAD"], padding="post",
                     dtype="long", truncating="post")

val_tags = pad_sequences([[tag2idx.get(l) for l in lab] for lab in val_labels],
                     maxlen=MAX_LEN, value=tag2idx["PAD"], padding="post",
                     dtype="long", truncating="post")

test_tags = pad_sequences([[tag2idx.get(l) for l in lab] for lab in test_labels],
                     maxlen=MAX_LEN, value=tag2idx["PAD"], padding="post",
                     dtype="long", truncating="post")

In [ ]:
attention_masks = [[float(i != 0.0) for i in ii] for ii in input_ids]

val_attention_masks = [[float(i != 0.0) for i in ii] for ii in val_input_ids]

test_attention_masks = [[float(i != 0.0) for i in ii] for ii in test_input_ids]

In [ ]:
tr_inputs = torch.tensor(input_ids)
tr_tags = torch.tensor(tags)
tr_masks = torch.tensor(attention_masks)

vl_inputs = torch.tensor(val_input_ids)
vl_tags = torch.tensor(val_tags)
vl_masks = torch.tensor(val_attention_masks)

tst_inputs = torch.tensor(test_input_ids)
tst_tags = torch.tensor(test_tags)
tst_masks = torch.tensor(test_attention_masks)

In [ ]:
tr_data = TensorDataset(tr_inputs, tr_masks, tr_tags)
train_sampler = RandomSampler(tr_data)
train_dataloader = DataLoader(tr_data, sampler=train_sampler, batch_size=bs)

valid_data = TensorDataset(vl_inputs, vl_masks, vl_tags)
valid_sampler = SequentialSampler(valid_data)
valid_dataloader = DataLoader(valid_data, sampler=valid_sampler, batch_size=bs)

tst_data = TensorDataset(tst_inputs, tst_masks, tst_tags)
tst_sampler = SequentialSampler(tst_data)
tst_dataloader = DataLoader(tst_data, sampler=tst_sampler, batch_size=bs)

In [ ]:
from transformers import AutoModelForTokenClassification, AutoTokenizer, Trainer, TrainingArguments, AdamW

In [ ]:
model_name = 'bert-base-multilingual-cased'
model = AutoModelForTokenClassification.from_pretrained(model_name, num_labels=len(tag2idx),
    output_attentions = False,
    output_hidden_states = False
)

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model.cuda();

In [ ]:
FULL_FINETUNING = True
if FULL_FINETUNING:
    param_optimizer = list(model.named_parameters())
    no_decay = ['bias', 'gamma', 'beta']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.0}
    ]
else:
    param_optimizer = list(model.classifier.named_parameters())
    optimizer_grouped_parameters = [{"params": [p for n, p in param_optimizer]}]

optimizer = AdamW(
    optimizer_grouped_parameters,
    lr=3e-5,
    eps=1e-8
)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
!pip install seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=2c6474f7b25d68315105cdd023e4f47669355f78f90537014d4d5b2832af0590
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


In [ ]:
from seqeval.metrics import f1_score, accuracy_score

In [ ]:
from tqdm import tqdm, trange

import numpy as np

In [ ]:
from transformers import get_linear_schedule_with_warmup

epochs = 10
max_grad_norm = 1.0

# Total number of training steps is number of batches * number of epochs.
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)

In [ ]:
## Store the average loss after each epoch so we can plot them.
loss_values, validation_loss_values = [], []

for _ in trange(epochs, desc="Epoch"):
    # ========================================
    #               Training
    # ========================================
    # Perform one full pass over the training set.

    # Put the model into training mode.
    model.train()
    # Reset the total loss for this epoch.
    total_loss = 0

    # Training loop
    for step, batch in enumerate(train_dataloader):
        # add batch to gpu
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        # Always clear any previously calculated gradients before performing a backward pass.
        model.zero_grad()
        # forward pass
        # This will return the loss (rather than the model output)
        # because we have provided the `labels`.
        outputs = model(b_input_ids, token_type_ids=None,
                        attention_mask=b_input_mask, labels=b_labels)
        # get the loss
        loss = outputs[0]
        # Perform a backward pass to calculate the gradients.
        loss.backward()
        # track train loss
        total_loss += loss.item()
        # Clip the norm of the gradient
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(parameters=model.parameters(), max_norm=max_grad_norm)
        # update parameters
        optimizer.step()
        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over the training data.
    avg_train_loss = total_loss / len(train_dataloader)
    print("Average train loss: {}".format(avg_train_loss))

    # Store the loss value for plotting the learning curve.
    loss_values.append(avg_train_loss)


    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    # Put the model into evaluation mode
    model.eval()
    # Reset the validation loss for this epoch.
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0
    predictions , true_labels = [], []
    for batch in valid_dataloader:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch

        # Telling the model not to compute or store gradients,
        # saving memory and speeding up validation
        with torch.no_grad():
            # Forward pass, calculate logit predictions.
            # This will return the logits rather than the loss because we have not provided labels.
            outputs = model(b_input_ids, token_type_ids=None,
                            attention_mask=b_input_mask, labels=b_labels)
        # Move logits and labels to CPU
        logits = outputs[1].detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        # Calculate the accuracy for this batch of test sentences.
        eval_loss += outputs[0].mean().item()
        predictions.extend([list(p) for p in np.argmax(logits, axis=2)])
        true_labels.extend(label_ids)

    eval_loss = eval_loss / len(valid_dataloader)
    validation_loss_values.append(eval_loss)
    print("Validation loss: {}".format(eval_loss))
    pred_tags = [tag_values[p_i] for p, l in zip(predictions, true_labels)
                                 for p_i, l_i in zip(p, l) if tag_values[l_i] != "PAD"]
    valid_tags = [tag_values[l_i] for l in true_labels
                                  for l_i in l if tag_values[l_i] != "PAD"]
    print("Validation Accuracy: {}".format(accuracy_score(pred_tags, valid_tags)))
    print()


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Average train loss: 0.40949008241295815


Epoch:  10%|█         | 1/10 [00:22<03:25, 22.88s/it]

Validation loss: 0.09492059797048569
Validation Accuracy: 0.85706340378198

Average train loss: 0.11213484464678913


Epoch:  20%|██        | 2/10 [00:45<03:00, 22.51s/it]

...

Validation loss: 0.06176472455263138
Validation Accuracy: 0.9315906562847609

Average train loss: 0.01651046145707369


Epoch:  90%|█████████ | 9/10 [03:28<00:23, 23.37s/it]

Validation loss: 0.06033494230359793
Validation Accuracy: 0.9332591768631813

Average train loss: 0.014834507848718204


Epoch: 100%|██████████| 10/10 [03:52<00:00, 23.20s/it]

Validation loss: 0.06128085404634476
Validation Accuracy: 0.9343715239154616



In [ ]:
model.eval()

predicted_labels = []
true_labels = []

# Iterate through the test_dataloader to get the predictions
# Using the GPU
for batch in tst_dataloader:
    batch = tuple(t.to(device) for t in batch)
    inputs = {'input_ids': batch[0], 'attention_mask': batch[1], 'labels': batch[2]}

    # Disabling gradient calculation for evaluation
    with torch.no_grad():
        outputs = model(**inputs)

    # Extracting the predictions and also the true labels for the test data
    predicted_labels.extend(outputs.logits.argmax(dim=-1).tolist())
    true_labels.extend(inputs['labels'].tolist())

In [ ]:
print(predicted_labels)
print(true_labels)

[[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7], [0, 0, 0, 0, 0, 1, 1, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 

In [ ]:
i = 0
while i < len(true_labels):
  if len(true_labels[i]) != len(predicted_labels[i]):
    print('issue')
    i += 1
  i += 1

In [ ]:
preds = []
trues = []

for sentences in true_labels:
  t = []
  for vals in sentences:
    if vals != 7:
      t.append(vals)
  trues.append(t)

for sentences in predicted_labels:
  p = []
  for vals in sentences:
    if vals != 7:
      p.append(vals)
  preds.append(p)

print(trues)
print(preds)

[[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 1, 1, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 1, 1, 1, 2, 2, 2, 2, 2, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 3, 3, 4, 4, 4, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 5, 5, 6, 6, 6, 6, 6, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [ ]:
print(test_tokenized_texts)
print(test_labels)
print(preds)

[['Mar', 'a', 't', '##ch', '##ít', '##ear', 'do', 'She', '##osa', '##mh', 'Mac', 'G', '##rian', '##na', 'é', 'cai', '##th', '##fi', '##dh', 'an', 't', '-', 'ea', '##la', '##ío', '##ntó', '##ir', 'an', 'sola', '##s', 'a', 'tha', '##bh', '##air', '##t', 'don', 'sao', '##l', 'agus', 'di', '##úl', '##tú', 'do', 'chat', '##h', '##ú', 'sin', 'na', 'tr', '##uai', '##ll', '##íocht', '##a', 'a', 'chu', '##irea', '##nn', 'an', 'sao', '##l', 'ina', 'cho', '##s', '##án', '.'], ['D', '##hi', '##úl', '##tai', '##gh', 'Johna', '##tha', '##n', 'Cape', 'agus', 'c', '##úp', '##la', 'foi', '##ls', '##ithe', '##oir', 'eile', 'an', 'lea', '##bhar', 'a', 'f', '##ho', '##ils', '##i', '##ú', 'agus', 'bhí', 'Naomi', 'le', 'cea', '##nga', '##l', 'go', 'raibh', 's', '##rian', 'a', 'chur', 'ar', 'a', 'cui', '##d', 'tua', '##iri', '##mí', '##ocht', '##a', '.'], ['G', '##rian', '##gh', '##rai', '##f', 'le', 'Mai', '##dh', '##c', 'Ó', 'Sea', '##chn', '##asa', '##í', '.'], ['Tag', '##ann', 'a', 'rá', '##ite', '##as',

In [ ]:
i = 0
while i < len(preds):
  if len(test_labels[i]) != len(preds[i]):
    print(f'issue: {i} {len(test_labels[i])}, {len(preds[i])}')
    print(f'{test_tokenized_texts[i]}\n{test_labels[i]}\n{preds[i]}')
    i += 1
  i += 1

In [ ]:
import pandas as pd

label_map = {
    0: 'O',
    1: 'B-PER',
    2: 'I-PER',
    3: 'B-LOC',
    4: 'I-LOC',
    5: 'B-ORG',
    6: 'I-ORG'
}

# Initialise an empty list to store DataFrames
dfs = []

# Iterate through the tokens, labels, and predictions
for tokens, labels, predictions in zip(test_tokenized_texts, test_labels, preds):
    # Create a DataFrame from the current sublist
    temp_df = pd.DataFrame({'Word': tokens, 'POS': 'X', 'True': labels, 'Predicted': [label_map[pred] for pred in predictions]})
    # Append the DataFrame to the list
    dfs.append(temp_df)
    # Add an empty row as a DataFrame to the list
    dfs.append(pd.DataFrame({'Word': [''], 'POS': [''], 'True': [''], 'Predicted': ['']}))

# Concatenate the DataFrames along the rows axis
df = pd.concat(dfs, ignore_index=True)

print(df)

        Word POS   True Predicted
0        Mar   X      O         O
1          a   X      O         O
2          t   X      O         O
3       ##ch   X      O         O
4       ##ít   X      O         O
...      ...  ..    ...       ...
7930      Ga   X  B-LOC     B-LOC
7931  ##elta   X  B-LOC     B-LOC
7932   ##cht   X  B-LOC     B-LOC
7933     ann   X      O         O
7934                             

[7935 rows x 4 columns]


In [ ]:
print(df.head(80))

    Word POS True Predicted
0    Mar   X    O         O
1      a   X    O         O
2      t   X    O         O
3   ##ch   X    O         O
4   ##ít   X    O         O
..   ...  ..  ...       ...
75  agus   X    O         O
76     c   X    O         O
77  ##úp   X    O         O
78  ##la   X    O         O
79   foi   X    O         O

[80 rows x 4 columns]


In [ ]:
conll_format = ""

for index, row in df.iterrows():
    text = row['Word']
    pos = row['POS']
    tag = row['True']
    mapped_tag = row['Predicted']

    conll_format += f"{text}\t{pos}\t{tag}\t{mapped_tag}\n"


with open('/content/mBERT_V2_eval.conll', 'w') as f:
      f.write(conll_format)